In [2]:
import torch
import torch.nn.functional as F
import sys
import os
current_directtory = os.getcwd()
sys.path[0]=os.path.abspath(os.path.join(current_directtory, "..", "charlm"))
from ngram import CharNGram
from charlm import CharLM
import random
import matplotlib.pyplot as plt

In [3]:
file = "names"

# Load data
with open(f"../data/input/{file}.txt", "r") as f:
#     names = list(set([i for i in f.read().splitlines()]))
    names = f.read().splitlines()

random.shuffle(names)

# Calculate the indices for splitting
total = len(names)
train_size = int(0.8 * total)
dev_size = int(0.1 * total)

# Split the names
names_train = names[:train_size]
names_dev = names[train_size:train_size + dev_size]
names_test = names[train_size + dev_size:]

In [4]:
# Acá ya no usamos cadenas de ngrams sino que usamos una representación continua de los caracteres
# y la unidad mínima ya no es es el n-gram sino el caracter, y se hacen append de la cantidad de caracteres
# que deseemos usar como predictor. Entonces, el universo que queremos representar es el universo de átomos (en
# este caso caracteres). Así, la matriz que representa los inputs crece hacia la izquierda (en columnas)
# y no hacia abajo (filas) como era en el caso de ngrams. Esto porque una cadena de strings ahora se representa
# como una cadena de vectores embeddings (más columnas) y no como una nueva fila

In [5]:
model = CharLM(context_length=3)

In [6]:
X_train, y_train, X_dev, y_dev, X_test, y_test= model.get_formatted_tensors(train = names_train,
                                                                            dev = names_dev,
                                                                            test = names_test)

In [7]:
model.fit(X_train, y_train, 
          neurons_per_layer = [100, 100,len(model.char_universe)], 
          activations = ["tanh", "tanh", "softmax"], 
          batch_size_percentage = 0.01,
          size_of_embeddings = 40, 
          epochs = 200000, 
          learning_rate = 0.1)

100%|██████████████████████████████████| 200000/200000 [04:31<00:00, 736.13it/s]


In [8]:
model.final_training_loss

2.0253448486328125

In [9]:
model.calculate_loss(X_test, y_test)

2.1506335735321045

In [10]:
model.calculate_loss(X_dev, y_dev)

2.156299114227295

In [34]:
words = model.generate_words(100)